In [2]:
import pandas as pd
import numpy as np
import ast

In [3]:
movies = pd.read_csv("./tmdb_5000_movies.csv")
credits = pd.read_csv("./tmdb_5000_credits.csv")

In [5]:
movies.shape

(4803, 20)

In [6]:
credits.shape

(4803, 4)

* Checking Null VALUES

In [ ]:
movies.isnull().sum()

In [ ]:
credits.isnull().sum()

# merge the two data frames

In [8]:
movies = movies.merge(credits,on='title')

# Selecting coloumns for recomendations

1. genres
2. homepage
3. movie_id
4. keywords
5. original_langauge
6. title
7. overview
8. cast
9. crew

In [10]:
movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]

# Preprocessing

In [ ]:
movies.isnull().sum()

In [12]:
movies.dropna(inplace=True)

* Preprocessing the generes

* '[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'
* [Action,Adventure,Fantasy,Science Fiction]

In [13]:
def preprocessGeneres(expression):
    li = ast.literal_eval(expression)
    l = []
    for ele in li:
        l.append(ele['name'])
    return l

In [14]:
movies['genres'] = movies['genres'].apply(preprocessGeneres)

* preprocessing the keywords

In [15]:
movies['keywords'] = movies['keywords'].apply(preprocessGeneres)

* prepocessing Cast

In [17]:
l = ast.literal_eval(movies['cast'][0])[:3]

In [18]:
def preprocessCast(obj):
    lt = ast.literal_eval(obj)[:3] # top 3 actors
    l = []
    for i in lt:
        l.append(i['name'])
    return l

In [19]:
movies['cast'] = movies['cast'].apply(preprocessCast)

* preprocessing Directors

In [20]:
def get_Directors(obj):
    directors = []
    for i in ast.literal_eval(obj):
        if(i['job'] == "Director"):
            directors.append(i['name'])
            break
    return directors

In [21]:
movies['crew'] = movies['crew'].apply(get_Directors)

* preprocessing overview

In [23]:

movies['overview'] = movies['overview'].apply(lambda x: x.split(' '))

In [24]:
movies['genres'] = movies['genres'].apply(lambda x : [i.replace(" ","") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x : [i.replace(" ","") for i in x])
movies['cast'] = movies['cast'].apply(lambda x : [i.replace(" ","") for i in x])
movies['crew'] = movies['crew'].apply(lambda x : [i.replace(" ","") for i in x])

In [25]:
movies['tags'] = movies['overview'] + movies['genres']  + movies['keywords']  + movies['cast'] + movies['crew']

In [26]:
new_df = movies[['movie_id','title','tags']]

In [ ]:
new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))

In [ ]:
new_df['tags'] = new_df['tags'].apply(lambda x : x.lower())

# text vectorization
* it gives the closest movies

In [29]:
from sklearn.feature_extraction.text import CountVectorizer
countVectorizer = CountVectorizer(max_features = 5000,stop_words = 'english')

In [30]:
new_df.shape


(4806, 3)

* reference https://www.geeksforgeeks.org/using-countvectorizer-to-extracting-features-from-text/

In [31]:
vectors = countVectorizer.fit_transform(new_df['tags']).toarray()

In [ ]:
# Frequency of each word in the dataset
countVectorizer.vocabulary_ 

In [33]:
len(countVectorizer.get_feature_names_out())

5000

In [ ]:
for i in countVectorizer.get_feature_names_out():
    print(i)

* Steming
1. ["loved","loving","love"]
2. Stem words => ["love","love","love"]

In [35]:
import nltk

In [36]:
from nltk.stem import PorterStemmer

In [37]:
stemmer = PorterStemmer()

In [38]:
def stemming(text):
    y = []
    for word in text.split():
        y.append(stemmer.stem(word))
    return " ".join(y)

In [ ]:
new_df['tags'] = new_df['tags'].apply(stemming)

In [40]:
from sklearn.metrics.pairwise import cosine_similarity

In [41]:
similarity = cosine_similarity(vectors)

In [42]:
def recommend(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    
    distances = similarity[movie_index]
    l = []
    for i in enumerate(distances):
        l.append(i)
    movies_list = sorted(l,reverse=True,key = lambda x: x[1])[1:6]
    recomended_movies_index = []
    for i in movies_list:
        recomended_movies_index.append(i[0])
        print(new_df.iloc[i[0]].title)
    
    return recomended_movies_index
    


In [43]:
recommend("Batman Begins")

The Dark Knight
The Dark Knight Rises
Batman
Batman & Robin
Batman


[65, 3, 1361, 210, 1360]

* Dumping the model using Pickle


In [45]:
import pickle

In [ ]:
pickle.dump(new_df,open('movies.pkl','wb'))

In [ ]:
pickle.dump(new_df.to_dict(),open('movies_dict.pkl','wb'))

In [ ]:
pickle.dump(similarity,open('similarity.pkl','wb'))

In [ ]:
import requests

In [ ]:

def fetchPosters(movie_id):
    response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key=db1bfac935013ecbf5f43957814edb44'.format(movie_id))
    data = response.json()
    return "https://image.tmdb.org/t/p/w500"+data['poster_path']


In [ ]:
l = recommend("Avatar")

Titan A.E.
Small Soldiers
Independence Day
Ender's Game
Aliens vs Predator: Requiem
